In [2]:
import pandas as pd
pd.set_option('display.max_rows', 10)

description_col = 'Security Description'
security_col = 'Security No.'
holding_col = 'Holding Jun24'
value_col = 'Market Value'
symbol_col = 'Symbol'
percent_col = '% of Fund'
sp_col = 'Share Price'

In [3]:
jema_symbols = pd.read_csv('./jema_may_jun24_symbols.csv').dropna(subset=[symbol_col])
jema_symbols[holding_col] = jema_symbols[holding_col].str.replace(',', '').astype(float)
jema_symbols.reset_index(inplace=True, drop=True)
jema_symbols

,Security Description,Security No.,Symbol,Exchange,Conv Rate,Holding Jun24
0,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,1,81271.00
1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,1,32585.00
2,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,1,164763.00
3,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,1,514330.73
4,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,1,47354.00
...,...,...,...,...,...,...
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,1,1301425.00
98,TCS GROUP HOLDING PLC GDR,0H6380S,TCSG,ALOR,1,75226.00
99,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,1,4246655.00
100,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,1,275182.00


In [4]:
jema_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='June 2024', skiprows=9, usecols=range(5))
jema_data = jema_data.dropna()
jema_data = jema_data.drop(columns=[description_col, 'Holding', percent_col])
jema_data[value_col] = jema_data[value_col].astype(float)
jema_data

,Market Value,Security No.
1,628932.02,BSHYYN1
2,559299.16,B12LZH9
3,548770.63,6606996
4,514330.73,5819115
5,432459.13,B030GJ7
...,...,...
110,14658.68,0H5808S
111,4294.55,0H6372S
112,1900.14,0H6380S
113,1156.81,0H6365S


In [5]:
symbols_with_jun_vals = pd.merge(jema_symbols, jema_data, on=security_col, how='left')

holdings_to_sum_vals = {
    "LKOH": ("2H6442S", "0H5797S"),
    "GAZP": ("0H5796S", "0H6364S"),
    "SBER": ("0H5856S", "0H5855S"),
    "NVTK": ("2H6464S", "0H5828S"),
    "ROSN": ("0H5827S", "2H7674S"),
    "TATN": ("0H5822S", "0H5816S"),
}

for symbol, (sec_no_1, sec_no_2) in holdings_to_sum_vals.items():
    val_sum = jema_data.loc[jema_data[security_col] == sec_no_1][value_col].item() + jema_data.loc[jema_data[security_col] == sec_no_2][value_col].item()
    symbols_with_jun_vals.loc[symbols_with_jun_vals[symbol_col] == symbol, value_col] = val_sum
    display(symbols_with_jun_vals.loc[symbols_with_jun_vals[symbol_col] == symbol])


,Security Description,Security No.,Symbol,Exchange,Conv Rate,Holding Jun24,Market Value
63,LUKOIL PJSC TOTAL,NaN,LKOH,ALOR,1,624696.0,300684.88


,Security Description,Security No.,Symbol,Exchange,Conv Rate,Holding Jun24,Market Value
36,GAZPROM PJSC TOTAL,NaN,GAZP,ALOR,1,19591077.0,226545.49


,Security Description,Security No.,Symbol,Exchange,Conv Rate,Holding Jun24,Market Value
94,SBERBANK OF RUSSIA PJSC TOTAL,NaN,SBER,ALOR,1,16810961.0,211058.6


,Security Description,Security No.,Symbol,Exchange,Conv Rate,Holding Jun24,Market Value
89,NOVATEK PJSC TOTAL,NaN,NVTK,ALOR,1,1372062.0,160222.55


,Security Description,Security No.,Symbol,Exchange,Conv Rate,Holding Jun24,Market Value
92,ROSNEFT OIL CO PJSC TOTAL,NaN,ROSN,ALOR,1,3719692.0,107524.32


,Security Description,Security No.,Symbol,Exchange,Conv Rate,Holding Jun24,Market Value
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,1,1301425.0,44357.4


In [6]:
symbols_with_jun_vals[sp_col] = symbols_with_jun_vals[value_col] / symbols_with_jun_vals[holding_col]
symbols_with_jun_vals

,Security Description,Security No.,Symbol,Exchange,Conv Rate,Holding Jun24,Market Value,Share Price
0,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,1,81271.00,628932.02,7.738702
1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,1,32585.00,559299.16,17.164314
2,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,1,164763.00,548770.63,3.330667
3,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,1,514330.73,514330.73,1.000000
4,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,1,47354.00,432459.13,9.132473
...,...,...,...,...,...,...,...,...
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,1,1301425.00,44357.40,0.034084
98,TCS GROUP HOLDING PLC GDR,0H6380S,TCSG,ALOR,1,75226.00,1900.14,0.025259
99,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,1,4246655.00,39977.21,0.009414
100,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,1,275182.00,1156.81,0.004204


In [9]:
# TODO: check share price according to tvdatafeed, flag those with error > some val
symbols_with_jun_vals.to_csv(path_or_buf="../data/jema_symbols_with_vals_jun24.csv")